In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 24.0 from /usr/local/lib/python3.11/site-packages/pip (python 3.11)
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 141.1 MB/s eta 0:00:00
  Installing build dependencies ...   Running command pip subprocess to install build dependencies
    Using cached setuptools-69.5.1-py3-none-any.whl.metadata (6.2 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl.metadata (14 kB)
    Using cached cmake-3.29.2-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.1 kB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
    Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
    Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
    Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
  Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB

In [2]:
!pip install llama-cpp-python==0.1.78


Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install PyPDF2

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# For download the models
!pip install huggingface_hub

Defaulting to user installation because normal site-packages is not writeable


We can quantize the model using this library, but for practical purposes, it is better to use pre-quantized models. The resulting model is only compatible with libraries that support GGML.


```
# obtain the original LLaMA model weights and place them in ./models
ls ./models
65B 30B 13B 7B tokenizer_checklist.chk tokenizer.model

# install Python dependencies
python3 -m pip install -r requirements.txt

# convert the 7B model to ggml FP16 format
python3 convert.py models/7B/

# quantize the model to 4-bits (using q4_0 method)
./quantize ./models/7B/ggml-model-f16.bin ./models/7B/ggml-model-q4_0.bin q4_0

# run the inference
./main -m ./models/7B/ggml-model-q4_0.bin -n 128
```



In [5]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

First, we download the model

In [6]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Inference with llama-cpp-python

Loading the model

In [7]:
model_path

'/home/vscode/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin'

In [8]:
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window 4096
)

llama.cpp: loading model from /home/vscode/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 4096
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_int


For run in CPU
```
# CPU
from llama_cpp import Llama

lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    )
```



In [9]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

43

We will use this prompt.

In [10]:
# prompt = "Write a linear regression in python"
# prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

# USER: {prompt}

# ASSISTANT:
# '''

In [11]:

import pandas as pd

import pandas as pd

data = {
    'Question': [
        "what is the candidate name? answer with the name only",
        "what is the candidate degree? only answer with the degree for example: bachelor ..etc without the degree name",
        "How many years of experience the candidate have? answer as a number",
        "How many technical experience the candidate have? include only technical work experience answer as a number",
        "Did the candidate worked for a ministry, authority, or government agency in Saudi Arabia? format your answer to be yes or no",
        "Please review the candidate's resume and provide a list of their skills?",
        "Please review the candidate's resume and provide a list of their certificates?",
        "What is the candidate's email? answer with only the email",
        "What is the candidate's count of certificates? answer with a number only",
        "What is the candidate's count of skills? answer with a number only",
        "What is the candidate's moblie or phone? answer with numbe only",
        "What is the candidate's speciality? ",
        "What is the candidate's major? answer witht he major only",
        "What is the candidate's gender?",
        "What is the candidate's expected Nationality? ",
        "What is the candidate's Job Title? answer with short answer "



    ],
    'title': [
        "Applicant Name",
        "Education Level",
        "Experience",
        "Ex_tech",
        "EX_Gov",
        "Skills",
        "Certification",
        "Email",
        "Count_CER",
        "Count_Skill",
        "Contact No",
        "Speciality",
        "Major",
        "Gender",
        "Nationality",
        "job title"
    ]
}

questions = pd.DataFrame(data)

In [12]:
questions

,Question,title
0,what is the candidate name? answer with the na...,Applicant Name
1,what is the candidate degree? only answer with...,Education Level
2,How many years of experience the candidate hav...,Experience
3,How many technical experience the candidate ha...,Ex_tech
4,"Did the candidate worked for a ministry, autho...",EX_Gov
5,Please review the candidate's resume and provi...,Skills
6,Please review the candidate's resume and provi...,Certification
7,What is the candidate's email? answer with onl...,Email
8,What is the candidate's count of certificates?...,Count_CER
9,What is the candidate's count of skills? answe...,Count_Skill


In [13]:
# @title
!pip install PyPDF2

Defaulting to user installation because normal site-packages is not writeable


In [14]:
import os
import PyPDF2
import pandas as pd

# Specify the folder path containing the PDF files
folder_path = '/workspaces/hc-usecase/CVs'

# Initialize an empty dataframe
df = pd.DataFrame(columns=['Filename', 'Text'])

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(folder_path, filename)

        # Open the PDF file
        with open(file_path, 'rb') as file:
            # Create a PDF reader object
            reader = PyPDF2.PdfReader(file)

            # Extract text from each page of the PDF
            text = ''
            for page in range(len(reader.pages)):
                text += reader.pages[page].extract_text()

            # Create a new DataFrame with the filename and extracted text
            new_row = pd.DataFrame({'Filename': [filename], 'Text': [text]})

            # Append the new row to the DataFrame using concat
            df = pd.concat([df, new_row], ignore_index=True)

# Print the DataFrame
print(df)

  Filename                                               Text
0  CV1.pdf  Mohammad Ali  \nEmail: Mali@gmail.com  \nMobil...


In [15]:
df

,Filename,Text
0,CV1.pdf,Mohammad Ali \nEmail: Mali@gmail.com \nMobil...


In [16]:
import pandas as pd

# Assuming you have already created the dataframe 'df'

# Define a function to process each row
def process_row(row):
    content = row['Text']
    file = row['Filename']
    for index, x in questions.iterrows():
      question = x['Question']
      prompt_template=f'''The following content is a resume of a candidate

      resume: {content}
      answer the question: {question}
      Answer:
      '''
      response = lcpp_llm(
      prompt=prompt_template,
      max_tokens=256,
      temperature=0.05,
      top_p=0.95,
      repeat_penalty=1.2,
      top_k=50,
      stop = ['USER:'], # Dynamic stopping when such token is detected.
      echo=True # return the prompt
      )

      print(response["choices"][0]["text"].split("Answer:")[1].strip())
      df.loc[df['Filename'] == file, x['title']] = response["choices"][0]["text"].split("Answer:")[1].strip()


# Loop through the dataframe and call the process_row function for each row
for index, row in df.iterrows():
    process_row(row)


llama_print_timings:        load time = 549876.86 ms
llama_print_timings:      sample time =   113.32 ms /     5 runs   (   22.66 ms per token,    44.12 tokens per second)
llama_print_timings: prompt eval time = 738674.04 ms /   649 tokens ( 1138.17 ms per token,     0.88 tokens per second)
llama_print_timings:        eval time = 309421.76 ms /     4 runs   (77355.44 ms per token,     0.01 tokens per second)
llama_print_timings:       total time = 1048722.42 ms


Mohammad Ali


Llama.generate: prefix-match hit

llama_print_timings:        load time = 549876.86 ms
llama_print_timings:      sample time =    38.60 ms /     3 runs   (   12.87 ms per token,    77.71 tokens per second)
llama_print_timings: prompt eval time = 80926.14 ms /    25 tokens ( 3237.05 ms per token,     0.31 tokens per second)
llama_print_timings:        eval time = 158926.98 ms /     2 runs   (79463.49 ms per token,     0.01 tokens per second)
llama_print_timings:       total time = 240051.74 ms
Llama.generate: prefix-match hit


Bachelor



llama_print_timings:        load time = 549876.86 ms
llama_print_timings:      sample time =   103.28 ms /     3 runs   (   34.43 ms per token,    29.05 tokens per second)
llama_print_timings: prompt eval time = 76381.74 ms /    19 tokens ( 4020.09 ms per token,     0.25 tokens per second)
llama_print_timings:        eval time = 156337.04 ms /     2 runs   (78168.52 ms per token,     0.01 tokens per second)
llama_print_timings:       total time = 233148.76 ms


...

Llama.generate: prefix-match hit

llama_print_timings:        load time = 549876.86 ms
llama_print_timings:      sample time =    39.16 ms /     2 runs   (   19.58 ms per token,    51.07 tokens per second)
llama_print_timings: prompt eval time = 82843.69 ms /    21 tokens ( 3944.94 ms per token,     0.25 tokens per second)
llama_print_timings:        eval time = 77648.44 ms /     1 runs   (77648.44 ms per token,     0.01 tokens per second)
llama_print_timings:       total time = 160637.64 ms
Llama.generate: prefix-match hit


3



llama_print_timings:        load time = 549876.86 ms
llama_print_timings:      sample time =    68.81 ms /     2 runs   (   34.41 ms per token,    29.06 tokens per second)
llama_print_timings: prompt eval time = 87165.47 ms /    35 tokens ( 2490.44 ms per token,     0.40 tokens per second)
llama_print_timings:        eval time = 73491.44 ms /     1 runs   (73491.44 ms per token,     0.01 tokens per second)
llama_print_timings:       total time = 160912.07 ms


Yes


Llama.generate: prefix-match hit


In [ ]:
# df = df.astype(str)


In [ ]:
df

In [ ]:
import re

def extract_numbers(text):
    numbers = re.findall(r'\d+', text)
    return numbers[0]

df['Ex_tech'] = df['Ex_tech'].apply(extract_numbers)
df['Experience'] = df['Experience'].apply(extract_numbers)

In [ ]:
# Extract "yes" or "no" using regular expressions
df['EX_Gov'] = df['EX_Gov'].str.extract(r'(yes|no)', flags=re.IGNORECASE)



In [ ]:
df['Text']

In [ ]:
Original = df

In [ ]:
Original

In [ ]:
import pandas as pd

education_dict = {
    'highschool': 1,
    'diploma': 2,
    'bachelor': 3,
    'masters': 3,
    'phd': 4,
    'professor': 5
}

df['degree_score'] = 0

for degree, score in education_dict.items():
    df.loc[
        df['Education Level'].str.lower().str.contains(degree),
        'degree_score'
    ] = score



In [ ]:
df

In [ ]:
import pandas as pd

def assign_ranks(df, column_name,  chosen_value):
    df['Rank'] = 0
    value = df.loc[df[column_name] == chosen_value, 'Score'].values[0]
    df.loc[df['Score'] == value, 'Rank'] = 2
    df.loc[df['Score'] < value, 'Rank'] = 1
    df.loc[df['Score'] > value, 'Rank'] = 2
    print(value)
    return df

def assign_ranks_1(df, column_name,  chosen_value):
    df['Rank'] = 1
    df['Score'] = 1
    df.loc[df[column_name]==chosen_value, 'Rank'] = 2

    return df





In [ ]:
data = {
    'Degree': ['PhD', 'Master', 'Bachelor', 'Diploma', 'High Scool'],
    'Score': [5, 4, 3, 2,1]
}

# Create the DataFrame
Degree = pd.DataFrame(data)
Degree

In [ ]:
data = {
    'Experience': ["'10-11'", "'8-9'", "'6-7'", "'4-5'", "'2-3'","'0-1'"],
    'Score': [6,5,4,3,2,1]
}

# Create the DataFrame
Experience = pd.DataFrame(data)

In [ ]:
import pandas as pd

# Define the list of skills
skills = { 'skills':[
    'Programming Languages',
    'Database Management',
    'Cybersecurity',
    'Cloud Computing',
    'DevOps',
    'Financial Analysis',
    'Accounting Principles',
    'Budgeting and Forecasting',
    'Taxation',
    'Financial Modeling',
    'Patient Care',
    'Medical Knowledge',
    'Healthcare Technology',
    'Regulatory Compliance',
    'Technical Design',
    'Project Management',
    'Quality Control',
    'Problem-Solving',
    'Digital Marketing',
    'Market Research',
    'Brand Management',
    'Creative Design',
    'Operational Management',
    'Health and Safety',
    'Quality Assurance',
    'Construction Planning'
]}

# Create a DataFrame from the list
skills = pd.DataFrame(skills)

# Display the DataFrame
print(skills)

In [ ]:
# @title
import pandas as pd

# Define the list of certificates
certificates = { 'certificates' : [
    'Certified Information Systems Security Professional (CISSP)',
    'Microsoft Certified',
    'AWS Certified Solutions Architect',
    'Cisco Certified Network Associate (CCNA) / Cisco Certified Network Professional (CCNP)',
    'Certified ScrumMaster (CSM)',
    'Certified Public Accountant (CPA)',
    'Chartered Financial Analyst (CFA)',
    'Certified Management Accountant (CMA)',
    'Certified Internal Auditor (CIA)',
    'Registered Nurse (RN)',
    'Board Certified Behavior Analyst (BCBA)',
    'Certified Clinical Medical Assistant (CCMA)',
    'Certified Registered Nurse Anesthetist (CRNA)',
    'Professional Engineer (PE)',
    'Project Management Professional (PMP)',
    'LEED Accredited Professional (LEED AP)',
    'Google Analytics IQ Certification',
    'HubSpot Content Marketing Certification',
    'Certified Digital Marketing Professional (CDMP) by Digital Marketing Institute',
    'Certified Supply Chain Professional (CSCP)',
    'Certified in Production and Inventory Management (CPIM)',
    'OSHA Certification',
    'Certified Construction Manager (CCM)'
]}

# Create a DataFrame from the list
certificates = pd.DataFrame(certificates)

# Display the DataFrame
certificates

In [ ]:
import pandas as pd
from ipywidgets import widgets, Layout, Box
from IPython.display import display

# Define the columns for the DataFrame
columns = ['Criteria', 'Value', 'Score']

# Create an empty DataFrame with the defined columns
df = pd.DataFrame(columns=columns)


# Dropdown widget for the degree input
degree_options = Degree['Degree']
degree_dropdown = widgets.Dropdown(options=degree_options, description='Degree:', style={'description_width': 'initial'})

# Scroll input widget for the degree percentage
degree_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

# Dropdown widget for the main specialty input
# main_specialty_options = ['Option 1', 'Option 2', 'Option 3', 'Option 4']
# main_specialty_dropdown = widgets.Dropdown(options=main_specialty_options, description='Main Specialty:', style={'description_width': 'initial'})

# # Scroll input widget for the main specialty percentage
# main_specialty_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

# Dropdown widget for the main skills input
main_skills_options = skills['skills']
main_skills_dropdown = widgets.Dropdown(options=main_skills_options, description='Main Skills:', style={'description_width': 'initial'})

# Scroll input widget for the main skills percentage
main_skills_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

# Dropdown widget for the main certificate input
main_certificate_options = certificates['certificates']
main_certificate_dropdown = widgets.Dropdown(options=main_certificate_options, description='Main Certificate:', style={'description_width': 'initial'})

# Scroll input widget for the main certificate percentage
main_certificate_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

# Dropdown widget for the years of experience input
years_of_experience_options = Experience['Experience']
years_of_experience_dropdown = widgets.Dropdown(options=years_of_experience_options, description='Years of Experience:', style={'description_width': 'initial'})

# Scroll input widget for the years of experience percentage
years_of_experience_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

# Radio button widget for government experience
government_experience_radio = widgets.RadioButtons(options=['Yes', 'No'], description='Government Experience:', style={'description_width': 'initial'})

# Scroll input widget for the government experience percentage
government_experience_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

Job_textbox = widgets.Text(
    description='Job Tiltle:',
    style={'description_width': 'initial'}
)

# Scroll input widget for the government experience percentage
Job_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

Manage_textbox = widgets.Text(
    description='Manage Skill:',
    style={'description_width': 'initial'}
)

# Scroll input widget for the government experience percentage
Manage_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})


Tech_textbox = widgets.Text(
    description='Tech Skill:',
    style={'description_width': 'initial'}
)

# Scroll input widget for the government experience percentage
Tech_percentage = widgets.FloatSlider(value=0, min=0, max=100, step=1, description='Percentage:', style={'description_width': 'initial'})

# Button to add the data to the DataFrame
add_button = widgets.Button(description='Calculate Score', button_style='success', tooltip='Click to add the data')

# Output widget to display messages
output = widgets.Output()

# Function to handle button click event
def add_to_dataframe(b):
    global df  # Declare df as a global variable

    with output:
        # Clear previous outputs
        output.clear_output()

        # Add data to the DataFrame
        new_data = pd.DataFrame([
            {'Criteria': 'Degree', 'Value': degree_dropdown.value, 'Score': degree_percentage.value},
            # {'Criteria': 'Main Specialty', 'Value': main_specialty_dropdown.value, 'Score': main_specialty_percentage.value},
            {'Criteria': 'skills', 'Value': main_skills_dropdown.value, 'Score': main_skills_percentage.value},
            {'Criteria': 'certificate', 'Value': main_certificate_dropdown.value, 'Score': main_certificate_percentage.value},
            {'Criteria': 'Experience', 'Value': years_of_experience_dropdown.value, 'Score': years_of_experience_percentage.value},
            {'Criteria': 'Government Experience', 'Value': government_experience_radio.value, 'Score': government_experience_percentage.value},
            {'Criteria': 'Job Title', 'Value': Job_textbox.value, 'Score': Job_percentage.value},
            {'Criteria': 'Tech skill', 'Value': Tech_textbox.value, 'Score': Manage_percentage.value},
            {'Criteria': 'Manage Skill', 'Value': Manage_textbox.value, 'Score': Tech_percentage.value}

        ])

        # Append data to the DataFrame
        df = df.append(new_data, ignore_index=True)
        print('Data added successfully!')

# Assign the click event handler to the button
add_button.on_click(add_to_dataframe)

# Create a box layout for the input widgets
# input_box = Box(children=[
#     degree_dropdown, degree_percentage,
#     main_specialty_dropdown, main_specialty_percentage,
#     main_skills_dropdown, main_skills_percentage,
#     main_certificate_dropdown, main_certificate_percentage,
#     years_of_experience_dropdown, years_of_experience_percentage,
#     government_experience_radio, government_experience_percentage,
#     add_button
# ], layout=Layout(
#     display='flex',
#     flex_flow='column',
#     align_items='stretch',
#     width='50%'
# ))
# Arrange widgets in a vertical box
input_widgets = widgets.VBox([
    widgets.HBox([degree_dropdown, degree_percentage]),
    # widgets.HBox([main_specialty_dropdown, main_specialty_percentage]),
    widgets.HBox([main_skills_dropdown, main_skills_percentage]),
    widgets.HBox([main_certificate_dropdown, main_certificate_percentage]),
    widgets.HBox([years_of_experience_dropdown, years_of_experience_percentage]),
    widgets.HBox([government_experience_radio, government_experience_percentage]),
    widgets.HBox([Job_textbox, Job_percentage]),
    widgets.HBox([Manage_textbox, Manage_percentage]),
    widgets.HBox([Tech_textbox, Tech_percentage])
])

# Display the input widgets and output widget
display(input_widgets, add_button, output)

In [ ]:
candidates = Original# @title


In [ ]:
df

In [ ]:
df[df['Criteria']=='Tech skill']['Value']


In [ ]:
# df['Score'] = df['Score'].astype(int)


In [ ]:
# # @title
# Degree =  assign_ranks(Degree, 'Degree',df['Value'][0])
# skills =  assign_ranks_1(skills, 'skills',df['Value'][1])
# certificates =  assign_ranks_1(certificates, 'certificates',df['Value'][2])
# Experience =  assign_ranks(Experience, 'Experience',df['Value'][3])



In [ ]:
# # @title
# Degree

In [ ]:
# # @title
# skills

In [ ]:
# # @title
# certificates

In [ ]:
# # @title
# Experience


In [ ]:
# # @title
# import pandas as pd

# # Specify the path to the Excel file
# excel_file_path = 'output2.xlsx'

# # Read the Excel file into a DataFrame
# candidates = pd.read_excel(excel_file_path)

# # Print the DataFrame
# candidates.columns

In [ ]:
candidates

In [ ]:
df

In [ ]:
# # @title
# mult = df['Score'][2]
# candidates.columns
# # Merge the candidate DataFrame with the rank DataFrame based on the degree
# candidates = pd.merge(candidates, Degree, left_on='Education Level', right_on='Degree', how='left')
# candidates = candidates.rename(columns={'Score': 'Degree_Rank'})
# candidates = candidates.drop('Rank', axis=1)
# candidates['Degree_Rank'] = candidates['Degree_Rank'].astype(int) * mult



In [ ]:
# candidates['Public_Rank'] = candidates['EX_Gov'].apply(lambda x: 1 if x == 'Yes' else 0)


In [ ]:
# @title
candidates

In [ ]:
df

In [ ]:
content = candidates['Text']
file = candidates['Filename']
temp = df[df['Criteria']=='Manage Skill']['Value'].reset_index()
question = temp['Value'][0]
prompt_template=f'''The following content is a resume of a candidate

resume: {candidates['Text']}
answer the question: Does the candidate have skills related to: {question}
Answer:
'''
response = lcpp_llm(
prompt=prompt_template,
max_tokens=256,
temperature=0.05,
top_p=0.95,
repeat_penalty=1.2,
top_k=50,
stop = ['USER:'], # Dynamic stopping when such token is detected.
echo=True # return the prompt
)
print(response["choices"][0]["text"].split("Answer:")[1].strip())

In [ ]:
print(candidates['Text'])

In [ ]:
candidates

In [ ]:
content.str.len()

In [ ]:
x = df[df['Criteria']=='Tech skill']['Value'].reset_index()

In [ ]:
x['Value'][0]

In [ ]:
print(original)

In [ ]:
def evaluate(candidates , df):
    df['Score'] = df['Score'].astype(int)
    # @title
    Degree =  assign_ranks(Degree, 'Degree',df['Value'][0])
    skills =  assign_ranks_1(skills, 'skills',df['Value'][1])
    certificates =  assign_ranks_1(certificates, 'certificates',df['Value'][2])
    Experience =  assign_ranks(Experience, 'Experience',df['Value'][3])
    # ------- Education level
    mult = df['Score'][2]
    candidates = pd.merge(candidates, Degree, left_on='Education Level', right_on='Degree', how='left')
    candidates = candidates.rename(columns={'Score': 'Degree_Rank'})
    candidates = candidates.drop('Rank', axis=1)
    candidates['Degree_Rank'] = candidates['Degree_Rank'].astype(int) * mult
    #------ Public (Yes or No)
    candidates['Public_Rank'] = candidates['EX_Gov'].apply(lambda x: 1 if x == 'Yes' else 0)

    # @title
    # -------- Total years of experience
    mult = df['Score'][1]
    candidates.columns
    # Merge the candidate DataFrame with the rank DataFrame based on the degree
    candidates = pd.merge(candidates, Experience, left_on='Experience', right_on='Experience', how='left')
    candidates = candidates.rename(columns={'Rank': 'Experience_Rank'})
    candidates['Experience_Rank'] = candidates['Experience_Rank'].astype(int) * mult


    content = candidates['Text']
    file = candidates['Filename']
    temp = df[df['Criteria']=='Manage Skill']['Value'].reset_index()
    question = temp['Value'][0]
    prompt_template=f'''The following content is a resume of a candidate

    resume: {candidates['Text']}
    answer the question: Does the candidate have skills related to: {question}
    Answer:
    '''
    resp
    response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=256,
    temperature=0.05,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USER:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
    )
    print(response["choices"][0]["text"].split("Answer:")[1].strip())
    df.loc[df['Filename'] == file, x['title']] = response["choices"][0]["text"].split("Answer:")[1].strip()




In [ ]:
# Loop through the dataframe and call the process_row function for each row
for index, row in candidates.iterrows():
    process_row(row, df)

In [ ]:

def ask_question(question){
      prompt_template=f'''The following content is a resume of a candidate

      resume: {content}
      answer the question: {question}
      Answer:
      '''
      response = lcpp_llm(
      prompt=prompt_template,
      max_tokens=256,
      temperature=0.05,
      top_p=0.95,
      repeat_penalty=1.2,
      top_k=50,
      stop = ['USER:'], # Dynamic stopping when such token is detected.
      echo=True # return the prompt
      )
}

In [ ]:
df

In [ ]:
# @title
#-----------------------------------
# df['Score'] = df['Score'].astype(int)
mult = df['Score'][1]
candidates.columns
# Merge the candidate DataFrame with the rank DataFrame based on the degree
candidates = pd.merge(candidates, skills, left_on='Skills', right_on='skills', how='left')
candidates = candidates.rename(columns={'Rank': 'skills_Rank'})
candidates = candidates.drop('Rank', axis=1)
candidates['skills_Rank'] = candidates['skills_Rank'].astype(int) * mult

In [ ]:
# @title
#-----------------------------------
# df['Score'] = df['Score'].astype(int)
mult = df['Score'][3]
candidates.columns
# Merge the candidate DataFrame with the rank DataFrame based on the degree
candidates = pd.merge(candidates, certificates, left_on='Certification', right_on='certificates', how='left')
candidates = candidates.rename(columns={'Rank': 'certificates_Rank'})
candidates['certificates_Rank'] = candidates['certificates_Rank'].astype(int) * mult

In [ ]:
candidates

In [ ]:
Experience

In [ ]:
# @title
#-----------------------------------
# df['Score'] = df['Score'].astype(int)
mult = df['Score'][1]
candidates.columns
# Merge the candidate DataFrame with the rank DataFrame based on the degree
candidates = pd.merge(candidates, Experience, left_on='Experience', right_on='Experience', how='left')
candidates = candidates.rename(columns={'Rank': 'Experience_Rank'})
candidates['Experience_Rank'] = candidates['Experience_Rank'].astype(int) * mult

In [ ]:
# @title
candidates['Score'] = candidates['Public_Rank'] + candidates['Public_Rank'] + candidates['skills_Rank']+ candidates['certificates_Rank']+ candidates['Experience_Rank']
max = candidates['Score'].max()
candidates['Score'] = (candidates['Score'] / max)*100

In [ ]:
# @title
candidates['Score']

In [ ]:
# @title
import pandas as pd
import numpy as np

# Create a sample DataFrame
df = candidates

# Define a color map
cmap = 'coolwarm'  # You can choose any valid colormap name

# Apply color formatting to the 'Score' column
df_styled = df.style.background_gradient(cmap=cmap, subset=['Score'])

# Display the styled DataFrame
df_styled

In [ ]:
# @title
candidates.columns

In [ ]:
# @title
import pandas as pd
import plotly.graph_objects as go



df = candidates[[  'Email', 'Education Level',
       'Major', 'Technical Experience Years', 'Managerial Experience Years',
       'Total Experience Years', 'Public Sector Experience', 'Date', 'Gender',
       'File Name', 'Citizenship', 'Score'
       ]]
# Convert the DataFrame to a Plotly figure
fig = go.Figure(data=go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df[col] for col in df.columns],
               fill_color='lavender',
               align='left')
))

# Set the layout properties of the table
fig.update_layout(
    title='DataFrame',
    width=1200,
    height=300,
    margin=dict(l=20, r=20, t=40, b=20)
)

# Display the Plotly figure
fig.show()

In [ ]:
# @title
import plotly.graph_objects as go
import pandas as pd

# Example dataframe
data = candidates[[ 'Email',  'Score'
       ]]
df = pd.DataFrame(data)

# Normalizing the 'Score' column for color scale (0-1)
max_score = df['Score'].max()
min_score = df['Score'].min()
df['Score_normalized'] = (df['Score'] - min_score) / (max_score - min_score)

# Define color scale - you can adjust the colors and thresholds
color_scale = [[0, 'green'], [0.5, 'yellow'], [1, 'red']]

# Create the table
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns[:-1]),  # Exclude the normalized score from the table display
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df.Email, df.Score],
               fill_color=[['white']*len(df),  # Normal background color for non-Score columns
                           [f'rgba({int(255 * c)}, {int(255 * (1-c))}, 0, 0.6)' for c in df['Score_normalized']]],  # Color for Score column
               align='left'))
])

fig.show()


In [ ]:
# @title
